In [1]:
import tensorflow as tf
import keras.backend as K
from kerassurgeon import identify
from kerassurgeon.operations import delete_channels

import os
import cv2
import random
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageFilter

from model_res import mobilenet_res

random.seed(2021)

os.environ["CUDA_VISIBLE_DEVICES"]="4"

2022-01-18 14:23:52.371222: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.


In [2]:
def get_list_from_filenames(file_path):
    with open(file_path,'r',) as f:
        lines = [one_line.strip('\n') for one_line in f.readlines()]
    return lines

### conv1+conv2 conv3+conv4 
### conv5+conv6 conv7+conv8
layers_name = ['conv_1', 
               'conv_2',
               'conv_3',
               'conv_4',
               'conv_5',
               'conv_6',
               'conv_7',
               'conv_8',
               'block_8_conv_1',
               ]

In [3]:
model = mobilenet_res(channels=[12,12,24,24,36,36,48,48,128])
model.load_weights("./checkpoints/tmp_res/cp-0005.hdf5")
print(model.summary())

batch_size = 256
num_epochs = 5
initial_learning_rate = 0.001

checkpoint_path = "./checkpoints/tmp_vgg_pruning/cp-{epoch:04d}.hdf5"
LOG_DIR = "./checkpoints/tmp_vgg_pruning_fitlogs/"

train_image_arr = np.load("./val_list_image_array.npy")
train_label_arr = np.load("./val_list_one_hot.npy")

val_image_arr = np.load("./val_list_image_array.npy")
val_label_arr = np.load("./val_list_one_hot.npy")

train_dataset = tf.data.Dataset.from_tensor_slices((train_image_arr, train_label_arr))
val_dataset   = tf.data.Dataset.from_tensor_slices((val_image_arr, val_label_arr))

train_dataset = train_dataset.shuffle(buffer_size=20000)
# train_dataset = train_dataset.repeat(2)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset   = val_dataset.batch(batch_size)
val_dataset   = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

2022-01-18 14:23:53.857890: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-01-18 14:23:53.968537: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:84:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.635GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-01-18 14:23:53.968583: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-01-18 14:23:53.971050: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-01-18 14:23:53.973183: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2022-01-18 14:23:53.973527: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 50, 50, 12)   120         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 50, 50, 12)   48          conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 50, 50, 12)   156         batch_normalization[0][0]        
_______________________________________________________________________________________

In [4]:
all_mask = []    
for i in range(len(layers_name)):
    print('------',layers_name[i],'------')
    layer_conv = model.get_layer(name=layers_name[i])
    apoz_layer_conv = identify.get_apoz(model, layer_conv, train_image_arr)
    high_apoz_channels_conv = identify.high_apoz(apoz_layer_conv, "both")
    print("---mask---", high_apoz_channels_conv,"-------")
    all_mask.append(high_apoz_channels_conv)

------ conv_1 ------


2022-01-18 14:23:57.010312: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2022-01-18 14:23:58.554832: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


---mask--- [2] -------
------ conv_2 ------
---mask--- [ 5 10] -------
------ conv_3 ------
---mask--- [2 6 7 8] -------
------ conv_4 ------
---mask--- [ 3  8  9 17 21] -------
------ conv_5 ------
---mask--- [ 1  3 11 17 22 24 27 30 32] -------
------ conv_6 ------
---mask--- [ 0  1 17 24 27 29 35] -------
------ conv_7 ------
---mask--- [ 8 16 19 20 27 29 31 32 34 39] -------
------ conv_8 ------
---mask--- [ 0  4  8  9 10 12 21 27 42 43 44 46] -------
------ block_8_conv_1 ------
---mask--- [  1   4   9  11  21  23  32  33  36  42  46  50  51  53  65  69  71  74
  76  79  93  97 105 107 108 110 122 127] -------


### 手动选择需要删除的通道

In [5]:
conv1_2 = [5,10]
conv3_4 = [3,8,9,17,21]
conv5_6 = [0,1,17,24,27,29,35]
conv7_8 = [0,4,8,9,10,12,21,27,42,43,44,46]
block_8_conv_1 = [1,4,9,11,21,23,32,33,36,42,46,50,51,53,65,69,71,74,76,79,93,97,105,107,108,110,122,127]

In [6]:
conv1_2_mask = [i for i in range(12) if i not in conv1_2]
conv3_4_mask = [i for i in range(24) if i not in conv3_4]
conv5_6_mask = [i for i in range(36) if i not in conv5_6]
conv7_8_mask = [i for i in range(48) if i not in conv7_8]
block_8_conv_1_mask = [i for i in range(128) if i not in block_8_conv_1]

In [7]:
origin_weight = model.get_weights()
origin_layers = model.layers
new_weights = origin_weight

In [8]:
for i in range(len(origin_weight)):
    print(origin_weight[i].shape)

(3, 3, 1, 12)
(12,)
(12,)
(12,)
(12,)
(12,)
(1, 1, 12, 12)
(12,)
(12,)
(12,)
(12,)
(12,)
(3, 3, 12, 24)
(24,)
(24,)
(24,)
(24,)
(24,)
(1, 1, 24, 24)
(24,)
(24,)
(24,)
(24,)
(24,)
(3, 3, 24, 36)
(36,)
(36,)
(36,)
(36,)
(36,)
(1, 1, 36, 36)
(36,)
(36,)
(36,)
(36,)
(36,)
(3, 3, 36, 48)
(48,)
(48,)
(48,)
(48,)
(48,)
(1, 1, 48, 48)
(48,)
(48,)
(48,)
(48,)
(48,)
(1, 1, 48, 128)
(128,)
(1, 1, 128, 4)
(4,)


In [8]:
def change(current_shape, j, new_weights, origin_weight, i, conv1_2_mask, num=12):
    if (current_shape[j]==num and j==0) or (current_shape[j]==num and j==1):
        new_weights[i] = origin_weight[i][conv1_2_mask]
    if current_shape[j]==num and j==2:
        new_weights[i] = origin_weight[i][:,:,conv1_2_mask,:]
    if current_shape[j]==num and j==3:
        new_weights[i] = origin_weight[i][:,:,:,conv1_2_mask]
    
for i in range(len(origin_weight)):
    current_shape = new_weights[i].shape
    for j in range(len(current_shape)):
        change(current_shape, j, new_weights, origin_weight, i, conv1_2_mask, 12)
        change(current_shape, j, new_weights, origin_weight, i, conv3_4_mask, 24)
        change(current_shape, j, new_weights, origin_weight, i, conv5_6_mask, 36)
        change(current_shape, j, new_weights, origin_weight, i, conv7_8_mask, 48)
        change(current_shape, j, new_weights, origin_weight, i, block_8_conv_1_mask, 128)

In [9]:
new_model = mobilenet_res(channels=[10,10,19,19,29,29,36,36,100])
new_model.summary()

Model: "functional_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 100, 100, 1) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 50, 50, 10)   100         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 50, 50, 10)   40          conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 50, 50, 10)   110         batch_normalization_8[0][0]      
______________________________________________________________________________________

In [10]:
new_model.set_weights(new_weights)

In [12]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                            decay_steps=15,
                                                            decay_rate=0.95,
                                                            staircase=True
                                                            )

new_model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=initial_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
              )

checkpointer = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1,
                                                  )

reduce_lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)

HISTORY = new_model.fit(train_dataset, 
                    epochs=num_epochs,
                    validation_data=val_dataset,
                    callbacks=[checkpointer, reduce_lr, tensorboard_callback],
                    shuffle=True,
                    )

2022-01-18 13:57:37.947025: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-01-18 13:57:37.947107: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


Epoch 1/5
 1/41 [..............................] - ETA: 0s - loss: 0.8581 - accuracy: 0.9102WARNING:tensorflow:From /mnt/fu04/xueluoyang/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


2022-01-18 13:57:40.944482: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-01-18 13:57:40.944576: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


 2/41 [>.............................] - ETA: 1s - loss: 0.6503 - accuracy: 0.9277WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0129s vs `on_train_batch_end` time: 0.0399s). Check your callbacks.


2022-01-18 13:57:40.970947: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-01-18 13:57:40.975672: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: ./checkpoints/tmp_vgg_pruning_fitlogs/train/plugins/profile/2022_01_18_13_57_40
2022-01-18 13:57:40.977000: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to ./checkpoints/tmp_vgg_pruning_fitlogs/train/plugins/profile/2022_01_18_13_57_40/fu04-SYS-4028GR-TR.trace.json.gz
2022-01-18 13:57:40.991073: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: ./checkpoints/tmp_vgg_pruning_fitlogs/train/plugins/profile/2022_01_18_13_57_40
2022-01-18 13:57:40.994459: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for memory_profile.json.gz to ./checkpoints/tmp_vgg_pruning_fitlogs/train/plugins/profile/2022_01_18_13_57_

41/41 [==============================] - ETA: 0s - loss: 0.5845 - accuracy: 0.9124
Epoch 00001: saving model to ./checkpoints/tmp_vgg_pruning/cp-0001.hdf5
41/41 [==============================] - 3s 63ms/step - loss: 0.5845 - accuracy: 0.9124 - val_loss: 0.8090 - val_accuracy: 0.9125
Epoch 2/5
41/41 [==============================] - ETA: 0s - loss: 0.4657 - accuracy: 0.9125
Epoch 00002: saving model to ./checkpoints/tmp_vgg_pruning/cp-0002.hdf5
41/41 [==============================] - 2s 53ms/step - loss: 0.4657 - accuracy: 0.9125 - val_loss: 0.7152 - val_accuracy: 0.9125
Epoch 3/5
41/41 [==============================] - ETA: 0s - loss: 0.4627 - accuracy: 0.9125
Epoch 00003: saving model to ./checkpoints/tmp_vgg_pruning/cp-0003.hdf5
41/41 [==============================] - 2s 52ms/step - loss: 0.4627 - accuracy: 0.9125 - val_loss: 0.7132 - val_accuracy: 0.9125
Epoch 4/5
41/41 [==============================] - ETA: 0s - loss: 0.4593 - accuracy: 0.9125
Epoch 00004: saving model to ./c